In [ ]:
import os, sys, timeit, math, copy, random
sys.path.append("../input/timmmaster/")

import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
import torchvision
import cv2
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch import nn, optim
from torch.cuda import amp
import torch.nn.functional as F

import timm

from sklearn.model_selection import StratifiedKFold

import albumentations as A
from albumentations.pytorch import ToTensorV2

import optuna

import transformers

In [ ]:
# Config and seed

class Config:
    img_size = 224
    n_splits = 10
    seed = 42
    n_bins = 20 # for validation
    
    device = "cuda"
    batch_size = 32
    num_workers = 2
    
    inference_oof = False
    inference_test = True
    
      
cfg = Config

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    #torch.backends.cudnn.deterministic = True

seed_everything(cfg.seed)

In [ ]:
# Dataset

class ImgDataset(Dataset):
    def __init__(self, transform=None, folder = "train"):
        self.transform = transform
        
        # Load labels
        self.df = pd.read_csv(f"../input/petfinder-pawpularity-score/{folder}.csv")
        self.folder = folder
                 
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        id_ = self.df.iloc[idx]["Id"]
        image = cv2.imread(f"../input/petfinder-pawpularity-score/{self.folder}/{id_}.jpg")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float64)
        
        if self.folder == "train":
            label = self.df.iloc[idx]['Pawpularity']
        else:
            label = 0
        
        if self.transform is not None:
            image = self.transform(image = image)["image"]
        
        return id_, image, torch.from_numpy(np.array(label)).float()

test_transform224 = A.Compose([
    A.SmallestMaxSize(224),
    A.CenterCrop(224, 224),
    A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0),
    ToTensorV2()
])

test_transform384 = A.Compose([
    A.SmallestMaxSize(384),
    A.CenterCrop(384, 384),
    A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
            max_pixel_value=255.0),
    ToTensorV2()
])

val_dataset224 = ImgDataset(transform = test_transform224, folder= "train")
test_dataset224 = ImgDataset(transform = test_transform224, folder= "test")

val_dataset384 = ImgDataset(transform = test_transform384, folder= "train")
test_dataset384 = ImgDataset(transform = test_transform384, folder= "test")

In [ ]:
# Validation

def validation():
    df = pd.read_csv(f"../input/petfinder-pawpularity-score/train.csv")
    labels = pd.cut(np.array(df["Pawpularity"]), bins=cfg.n_bins, labels=False)

    splitter = StratifiedKFold(n_splits = cfg.n_splits, shuffle = True, random_state = cfg.seed)
    splits = splitter.split(labels, labels)
    return splits

In [ ]:
# Modelling

class BaseSwin(nn.Module):
    def __init__(self, name = "swin_large_patch4_window7_224", pretrained = True):
        super(BaseSwin, self).__init__()
        self.model = timm.create_model(name, pretrained=pretrained)
        #self.model.head = nn.Sequential(nn.Dropout(p =  dropout), nn.Linear(self.model.head.in_features, 1))
        self.model.head = nn.Linear(self.model.head.in_features, 1)

    def forward(self, x):
        x = self.model(x)
        return x.ravel()
    
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [ ]:
# Inference

def inference_bce_scaled(preds):
    # Inference function transforming model output to predictions
    return sigmoid(preds) * 100

def inference_oof(model_class, model_params, weights_path, dataset, inference_function, train_df, columnname = "model1"):
    splits = validation()
    
    device = cfg.device
    
    train_df[columnname] = -1
    
    for fold, (train_idx,val_idx) in enumerate(splits):
        print(f"Fold {fold + 1}", "\n")
        
        model = model_class(**model_params)
        model.to(device)
        
        filename = f'{weights_path}/fold_{fold}.pth'
        model.load_state_dict(torch.load(filename, map_location="cuda"))
        model.eval()
        
        val_sampler = SubsetRandomSampler(val_idx)
        dataloader = DataLoader(dataset, batch_size=cfg.batch_size, sampler=val_sampler, num_workers = cfg.num_workers, pin_memory = True)
        
        for id_, X, target in tqdm(dataloader):
            with torch.no_grad():
                X, target = X.to(device), target.to(device)

                output = model(X)
                preds = inference_function(output)
                
                train_df.loc[id_, columnname] = preds.detach().cpu().numpy()
                
def inference_test(model_class, model_params, weights_path, dataset, inference_function, submission_df, columnname = "model1"):
    dataloader = DataLoader(dataset, batch_size=32, num_workers = 2)
    df = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
    df.set_index("Id", inplace = True)
    device = cfg.device
    
    for fold in range(cfg.n_splits):
        df[fold] = -1
        print(f"Fold {fold + 1}", "\n")
        
        model = model_class(**model_params)
        model.to(device)
        
        filename = f'{weights_path}/fold_{fold}.pth'
        model.load_state_dict(torch.load(filename, map_location="cuda"))
        model.eval()
        
        for id_, X, target in tqdm(dataloader):
            with torch.no_grad():
                X, target = X.to(device), target.to(device)

                output = model(X)
                preds = inference_function(output)
                
                df.loc[id_, fold] = preds.detach().cpu().numpy()
    
    columnnames = [x for x in range(cfg.n_splits)]
    submission_df[columnname] = df[columnnames].mean(axis = 1)

In [ ]:
# OOF preds + blend
if cfg.inference_oof:
    train_df = pd.read_csv(f"../input/petfinder-pawpularity-score/train.csv")
    train_df.set_index("Id", inplace = True)

    # 224 model
    params_224 = {"name": "swin_large_patch4_window7_224", "pretrained" : False}
    inference_oof(BaseSwin, params_224, "../input/bce-large-1-epoch", val_dataset224, inference_bce_scaled, train_df, "224")

    # 384 model
    params_384 = {"name": "swin_large_patch4_window12_384", "pretrained" : False}
    inference_oof(BaseSwin, params_384, "../input/petfinder3842", val_dataset384, inference_bce_scaled, train_df, "384")
    
    # Ridge OOF blender
    from sklearn.linear_model import Ridge
    blender = Ridge(alpha = 0.1, fit_intercept = False)
    blender.fit(train_df[["224", "384"]], train_df["Pawpularity"])
    coef = blender.coef_
    
    from sklearn.metrics import mean_squared_error
    rmse_mean = mean_squared_error((train_df["224"] + train_df["384"]) / 2, train_df["Pawpularity"], squared = False)
    rmse_blend = mean_squared_error(coef[0]*train_df["224"] + coef[1]*train_df["384"], train_df["Pawpularity"], squared = False)
    
    print(coef, rmse_mean, rmse_blend)

In [ ]:
# Test prediction
if cfg.inference_test:
    submission_df = pd.read_csv(f"../input/petfinder-pawpularity-score/sample_submission.csv")
    submission_df.set_index("Id", inplace = True)
    
    # 224 model
    params_224 = {"name": "swin_large_patch4_window7_224", "pretrained" : False}
    inference_test(BaseSwin, params_224, "../input/bce-large-1-epoch", test_dataset224, inference_bce_scaled, submission_df, "224")

    # 384 model
    params_384 = {"name": "swin_large_patch4_window12_384", "pretrained" : False}
    inference_test(BaseSwin, params_384, "../input/petfinder3842", test_dataset384, inference_bce_scaled, submission_df, "384")
    
    submission_df["Pawpularity"] = submission_df["224"] * 0.4266 + submission_df["384"] * 0.5790
    submission_df.drop(columns = ["224", "384"], inplace = True)
    submission_df.to_csv(f'submission.csv')
    
    print(submission_df)